In [1]:
import os
import pandas as pd
import re
import requests
import numpy as np
import matplotlib.pyplot as plt
from time import time
from multiprocessing.pool import ThreadPool

def url_descarga(url):
    Nombre, url = url
    r = requests.get(url, stream = True)
    print(Nombre,'entro\n')
    with open(Nombre, 'wb') as f:

        for archivo in r:
            f.write(archivo)

# Crear Carpeta Datos_web donde guardar archivos descargados desde la ANH
path = os.getcwd()
print(path)
dir = os.path.join(path,"Datos_web")
if not os.path.exists(dir):
    os.mkdir(dir)

urls = [
("Datos_web/Produccion_Fiscalizada_Crudo_2013.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20fiscalizada%20de%20crudo%20a%c3%b1o%202013.xlsx"),
("Datos_web/Produccion_Fiscalizada_Crudo_2014.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20fiscalizada%20de%20crudo_2014_31122014.xlsx"),
("Datos_web/Produccion_Fiscalizada_Crudo_2015.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202015.xlsx"),
("Datos_web/Produccion_Fiscalizada_Crudo_2016.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20fiscalizada%20crudo%202016_18042018.xls"),
("Datos_web/Produccion_Fiscalizada_Crudo_2017.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Produccion-fiscalizada-crudo-2017.xlsx"),
("Datos_web/Produccion_Fiscalizada_Crudo_2018.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202018.xlsx"),
("Datos_web/Produccion_Fiscalizada_Crudo_2019.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202019-DIC.xlsx"),
("Datos_web/Produccion_Fiscalizada_Crudo_2020.xlsx", "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documentos%20compartidos/Producci%c3%b3n%20Fiscalizada%20Crudo%202020%20Agosto.xlsx")
]
#Descarga una por una
'''for x in urls:
    url_response (x)'''

ThreadPool(9).imap_unordered(url_descarga, urls)#descarga simultanea


C:\Users\jnina\Jupyter\HackathonOG


In [2]:
#Creear data frames con los datos de Produccion de Crudo: de 2018 a 2019
df_2020 = pd.read_excel('Datos_web/Produccion_Fiscalizada_Crudo_2020.xlsx')
df_2019 = pd.read_excel('Datos_web/Produccion_Fiscalizada_Crudo_2019.xlsx')
df_2018 = pd.read_excel('Datos_web/Produccion_Fiscalizada_Crudo_2018.xlsx')

In [3]:
# Nombres de columnas estan mal (se puede borrar)
df_2020.columns.values

array(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12'], dtype=object)

In [4]:
# Funcion para limpieza del archivo excel y poner en formato
def format_excel(df_excel_raw, year = 2020):
    df_excel_raw.dropna(inplace = True) #eliminar filas NO validas
    df_excel_raw.rename(columns=df_excel_raw.iloc[0],inplace = True) #reemplazar el nombre de las columnas
    df_excel_raw.drop(df_excel_raw.index[0], inplace = True)
    df_excel_raw.reset_index(drop = True, inplace = True) # reiniciar el indexado de las filas
    
    # llenar datos nulos para meses faltantes (si no tiene datos de produccion para cada mes)
    df_excel_raw.columns = df_excel_raw.columns.str.strip().str.lower() # convertir nombre de columnas a minusculas y sin espacios
    meses = ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
    
    for mes in meses:
        if (0 in np.core.defchararray.find( df_excel_raw.columns.values.astype(str), mes)):
            # reemplazar nombre de Mes a nombre corto (Enero --> ene)
            df_excel_raw.columns.values[[i for i,item in enumerate(df_excel_raw.columns.values) if mes in item]] = mes            
            
        else:
            # Si no hay datos para el mes, anadir una columna de ceros para ese mes
            df_excel_raw[mes] = pd.Series([0 for x in range(len(df_excel_raw.index))], index=df_excel_raw.index)
    
    # anadir columna con anho
    df_excel_raw['año'] = pd.Series([year for x in range(len(df_excel_raw.index))], index=df_excel_raw.index)
    return df_excel_raw

In [5]:
format_excel(df_2020,2020)
format_excel(df_2019,2019)
format_excel(df_2018,2018)
df_2020.head(5) #mostrar produccion de crudo - 2020

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949,0,0,0,0,2020
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08,0,0,0,0,2020
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73,0,0,0,0,2020
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94,0,0,0,0,2020
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,187.33,184.97,173.58,191.32,193.94,166.74,168.84,152.02,0,0,0,0,2020


In [6]:
# Unir produccion desde 2018 a 2020 en Un unico DF
df_2018_2020 = pd.concat([df_2020, df_2019, df_2018], ignore_index = True)

In [7]:
# Mostrar produccion de 2018 a 2020
df_2018_2020.head()

,departamento,municipio,operadora,contrato,campo,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic,año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949,0,0,0,0,2020
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08,0,0,0,0,2020
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73,0,0,0,0,2020
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94,0,0,0,0,2020
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,187.33,184.97,173.58,191.32,193.94,166.74,168.84,152.02,0,0,0,0,2020


In [8]:
# Exportar a .xlsx - produccion 2018 a 2020
df_2018_2020.to_excel('Datos_web\Produccion_Fiscalizada_Crudo_2018_2020.xlsx', index = False)

Datos_web/Produccion_Fiscalizada_Crudo_2014.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2020.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2013.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2017.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2019.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2018.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2015.xlsx entro

Datos_web/Produccion_Fiscalizada_Crudo_2016.xlsx entro

